In [10]:
import numpy as np
import pandas as pd
import csv
import ast
import collections
import matplotlib.pyplot as plt
import Levenshtein as lev
from fuzzywuzzy import fuzz 
import datetime
import matplotlib.pyplot as plt
import re
from urllib.parse import urlparse
from collections import Counter

from elsapy.elsclient import ElsClient
from elsapy.elsdoc import FullDoc, AbsDoc
from elsapy.elssearch import ElsSearch

import time # for sleep
from pybtex.database import parse_file, BibliographyData, Entry
import json
from elsapy.elsclient import ElsClient
from elsapy.elsdoc import AbsDoc
from elsapy.elssearch import ElsSearch

import geopandas

ModuleNotFoundError: No module named 'geopandas'

In [5]:
def AffiliationsFromScopusByDOI(client, doi):
    """obtain addiotional paper information from scopus by doi
    """
    doc_srch = ElsSearch("DOI("+doi+")",'scopus')
    doc_srch.execute(client, get_all = True)
    #print ("doc_srch has", len(doc_srch.results), "results.")
    #print(doc_srch.results)
    try:
        scopus_id=doc_srch.results[0]["dc:identifier"].split(":")[1]
        scp_doc = AbsDoc(scp_id = scopus_id)
        if scp_doc.read(client):
            # print ("scp_doc.title: ", scp_doc.title)
            scp_doc.write()   
        else:
            print ("Read document failed.")
        # print(scp_doc.data["affiliation"])
        return scp_doc.data
    except:
        return None

In [6]:
## Load configuration
con_file = open("config.json")
config = json.load(con_file)
con_file.close()
## Initialize client
client = ElsClient(config['apikey'])
# https://api.elsevier.com/content/search/scopus?query=DOI(10.1109/MCOM.2016.7509373)&apiKey=6d485ef1fe1408712f37e8a783a285a4

In [7]:
df_current_extra_info = pd.read_pickle('extra_info_CS5099.pkl')
df_current_extra_info

,affiliation,coredata
0,"[{'affiliation-city': None, 'affilname': 'Pata...","{'srctype': 'j', 'prism:issueIdentifier': '230..."
1,"[{'affiliation-city': 'New Delhi', 'affilname'...","{'srctype': 'j', 'eid': '2-s2.0-85083171050', ..."
2,None,None
3,"[{'affiliation-city': 'London', 'affilname': '...","{'srctype': 'j', 'eid': '2-s2.0-79953057246', ..."
4,"[{'affiliation-city': 'Baoding', 'affilname': ...","{'srctype': 'j', 'prism:issueIdentifier': '7',..."
...,...,...
2804,"[{'affiliation-city': 'Ottawa', 'affilname': '...","{'srctype': 'j', 'prism:issueIdentifier': '8',..."
2805,"[{'affiliation-city': 'Calgary', 'affilname': ...","{'srctype': 'j', 'eid': '2-s2.0-85096860901', ..."
2806,None,None
2807,"[{'affiliation-city': 'Madrid', 'affilname': '...","{'srctype': 'j', 'eid': '2-s2.0-85076479619', ..."


In [12]:
dict_extra_info = dict()
dict_extra_info[0] = AffiliationsFromScopusByDOI(client, '10.1016/S1525-1578(10)60571-5')
dict_extra_info[1] = AffiliationsFromScopusByDOI(client, 'dict()10.31729/jnma.5498')

In [13]:
df_extra_info_test = pd.DataFrame(dict_extra_info)
df_extra_info_test

,0,1
affiliation,"[{'affiliation-city': 'Boston', 'affilname': '...",None
coredata,"{'srctype': 'j', 'eid': '2-s2.0-23844497341', ...",None


In [14]:
df_extra_info_test_transposed = df_extra_info_test.T
df_extra_info_test_transposed

,affiliation,coredata
0,"[{'affiliation-city': 'Boston', 'affilname': '...","{'srctype': 'j', 'eid': '2-s2.0-23844497341', ..."
1,None,None


In [24]:
df_combined = pd.concat([df_current_extra_info, df_extra_info_test_transposed],ignore_index=True)
df_combined

,affiliation,coredata
0,"[{'affiliation-city': None, 'affilname': 'Pata...","{'srctype': 'j', 'prism:issueIdentifier': '230..."
1,"[{'affiliation-city': 'New Delhi', 'affilname'...","{'srctype': 'j', 'eid': '2-s2.0-85083171050', ..."
2,None,None
3,"[{'affiliation-city': 'London', 'affilname': '...","{'srctype': 'j', 'eid': '2-s2.0-79953057246', ..."
4,"[{'affiliation-city': 'Baoding', 'affilname': ...","{'srctype': 'j', 'prism:issueIdentifier': '7',..."
...,...,...
2806,None,None
2807,"[{'affiliation-city': 'Madrid', 'affilname': '...","{'srctype': 'j', 'eid': '2-s2.0-85076479619', ..."
2808,None,None
2809,"[{'affiliation-city': 'Boston', 'affilname': '...","{'srctype': 'j', 'eid': '2-s2.0-23844497341', ..."
